In [1]:
import asyncio
import os
from uuid import uuid4
from typing import Any, Dict, List, Mapping, Optional, cast

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing With Write Queue - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://dev.api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = ""  # Update to your API key

In [2]:
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun

In [3]:
class FakeLLM(LLM):
    """Fake LLM wrapper for testing purposes."""
    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "fake"
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        return "This is a fake response"

In [4]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("This is a test {input}")
model = FakeLLM()
chain = prompt | model

In [5]:
async def task():
    return await chain.ainvoke({"input": "test"})

async def chunked_gather(func, n, chunk_size):
    """Run func n times, chunk_size concurrently."""
    for _ in range(0, n, chunk_size):
        tasks = [func() for _ in range(chunk_size)]
        await asyncio.gather(*tasks)

await chunked_gather(task, 10000, 20)